# Challenging Fake Image Detection from GAN Models

#### Detecting fake or manipulated images in today's digital age has become increasingly challenging due to the advancements in Generative Adversarial Networks (GANs). These AI-powered tools have made it easier than ever to create convincing fake images that can deceive both human observers and traditional image analysis techniques. The problem at hand is to develop a robust and effective fake image detection system that can differentiate between genuine and manipulated images generated by GAN models.

### By: Nithish B A

In [1]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim

### Adjustable Parameters

In [2]:
train_dir = "./train/" 
test_directory = "./test/"
batch_size = 64
epoaches = 20

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [4]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Assuming RGB images
])

In [5]:
train_dataset = ImageFolder(train_dir, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = ImageFolder(test_directory, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
class FakeImageDetectionCNN(nn.Module):
    def __init__(self):
        super(FakeImageDetectionCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = FakeImageDetectionCNN().to(device)

In [7]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(epoaches):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels.float().view(-1, 1))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

print("Finished Training")

Epoch 1, Loss: 0.3035092779261823
Epoch 2, Loss: 0.19636853401306648
Epoch 3, Loss: 0.16340265749111743
Epoch 4, Loss: 0.1394244196252112
Epoch 5, Loss: 0.11920026089323975
Epoch 6, Loss: 0.10218614367357028
Epoch 7, Loss: 0.08420496053757744
Epoch 8, Loss: 0.07170687709748506
Epoch 9, Loss: 0.05809437793096193
Epoch 10, Loss: 0.047244171460170445
Epoch 11, Loss: 0.03982049556745792
Epoch 12, Loss: 0.0353339862376952
Epoch 13, Loss: 0.02856586320640493
Epoch 14, Loss: 0.02644139255990792
Epoch 15, Loss: 0.023744815705127785
Epoch 16, Loss: 0.02281674137103447
Epoch 17, Loss: 0.01744458377647448
Epoch 18, Loss: 0.01911349144392356
Epoch 19, Loss: 0.016407861690392156
Epoch 20, Loss: 0.018975375909528826
Finished Training


In [8]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        predicted = (outputs >= 0.5).view(-1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on Test Dataset: {100 * correct / total:.2f}%")

Accuracy on Test Dataset: 93.57%
